In [2]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
#loading the data
train_data = pd.read_csv("dataset/train.csv")
test_data = pd.read_csv("dataset/test.csv")

In [4]:
train_data.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [5]:
#Encoding the categorical values to numerical values
le = LabelEncoder()
train_data['Class']=le.fit_transform(train_data['Class'])
# train_data['Class']

In [6]:
#Separating the x_train and y_train
x_train=[]
y_train=[]
i=0
for img in train_data['Image']:
    path = os.path.join("dataset/Train Images",img)
    train_img = cv2.imread(path)
    train_img=cv2.resize(train_img,(200,200))
    train_img = train_img.astype('float32')
    x_train.append(train_img)
    y_train.append(train_data['Class'][i])
    i=i+1
# print(x_train[0])


In [ ]:
x_test=[]
for img in test_data['Image']:
    path = os.path.join("dataset/Test Images",img)
    test_img = cv2.imread(path)
    test_img=cv2.resize(test_img,(200,200))
    test_img = test_img.astype('float32')
    x_test.append(test_img)

In [7]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(5983, 200, 200, 3)
(5983,)
(3219, 200, 200, 3)


In [1]:
import keras
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten, Conv2D, Dropout, Activation, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

Using TensorFlow backend.


In [9]:
num_classes = 4
batch_size = 128
epochs = 10

In [ ]:
#creating the model
model = Sequential()
model.add(VGG16(include_top=False, pooling='avg', weights='imagenet', input_shape=(200,200,3)))
model.add(Dense(256,activation='relu'))
model.add(Dense(num_classes, activation = "softmax"))
model.layers[0].trainable = False
model.summary()

In [ ]:
#Data Augmentation
datagen = ImageDataGenerator(
      featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [ ]:
#compiling the model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
model.fit_generator(datagen.flow(x_train,to_categorical(y_train,4), batch_size=batch_size, shuffle = False),
                              epochs = epochs,                          
                              callbacks=[learning_rate_reduction], workers=4)
# model.fit(x_train, to_categorical(y_train,4), batch_size=batch_size, epochs=epochs, callbacks=[learning_rate_reduction])

In [ ]:
y_test = model.predict(x_test)
label = [np.argmax(i) for i in y_test]
y_test = le.inverse_transform(label)
print(y_test)
submission = pd.DataFrame({'Image':test_data.Image, 'Class':y_test})
submission.to_csv('submission.csv',index=False)
